In [0]:
from delta.tables import *

stage_table = "incremental_load.default.orders_stage"
target_table = "incremental_load.default.orders_target"

In [0]:
stage_df = spark.read.table(stage_table)

In [0]:
if not spark._jsparkSession.catalog().tableExists(target_table):
    stage_df.write.format("delta").saveAsTable(target_table)

else:
    target_table = DeltaTable.forName(spark, target_table)
    
    mergeCondition = "stage.tracking_num = target.tracking_num"

    target_table.alias("target").merge(
            stage_df.alias("stage"),
            mergeCondition
        )\
        .whenMatchedDelete() \
        .execute()
    
    stage_df.write.format("delta").mode("append").saveAsTable(target_table)